In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 20886 (delta 22), reused 18 (delta 7), pack-reused 20830
Receiving objects: 100% (20886/20886), 16.46 MiB | 4.35 MiB/s, done.
Resolving deltas: 100% (15241/15241), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 

In [187]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [188]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [189]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 4                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 3
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [190]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [191]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [192]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [193]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [194]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [195]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [196]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [197]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [198]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [199]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [200]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [201]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2.rename(columns = {'T' : 'Temp'}, inplace = True)
dataset2

,DHI,DNI,WS,RH,Temp,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [202]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,Temp,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [203]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [204]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [205]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'Temp','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['RH', 'Temp', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,RH,Temp,SH,sin_time,cos_time,GHI,TARGET
0,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


In [206]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset6)
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [207]:
dataset7 = train_to_supervised(temp_X,temp_y, n_days)
dataset7

,RH(t-192),Temp(t-192),SH(t-192),sin_time(t-192),cos_time(t-192),GHI(t-192),TARGET(t-192),RH(t-191),Temp(t-191),SH(t-191),sin_time(t-191),cos_time(t-191),GHI(t-191),TARGET(t-191),RH(t-190),Temp(t-190),SH(t-190),sin_time(t-190),cos_time(t-190),GHI(t-190),TARGET(t-190),RH(t-189),Temp(t-189),SH(t-189),sin_time(t-189),cos_time(t-189),GHI(t-189),TARGET(t-189),RH(t-188),Temp(t-188),SH(t-188),sin_time(t-188),cos_time(t-188),GHI(t-188),TARGET(t-188),RH(t-187),Temp(t-187),SH(t-187),sin_time(t-187),cos_time(t-187),...,cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),Temp(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),Temp(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),Temp(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),Temp(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),Temp(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
192,0.665404,0.129630,0.083333,0.500000,1.000000,0.0,0.0,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,...,0.853553,0.0,0.0,0.649497,0.259259,0.083333,0.195619,0.896677,0.0,0.0,0.695704,0.259259,0.083333,0.250000,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.0,0.0
193,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,...,0.896677,0.0,0.0,0.695704,0.259259,0.083333,0.250000,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.0,0.0
194,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,...,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0,0.0,0.0
195,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,...,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0,0.698517,0.259259,0.083333,0.629410,0.982963,0.0,0.0,0.0,0.0
196,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,0.0,0.0,0.725246,0.148148,0.083333,0.961940,0.691342,...,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0,0.698517,0.259259,0.083333,0.629410,0.982963,0.0,0.0,0.698193,0.259259,0.083333,0.691342,0.961940,0.0,0.0,0.0,0.0
...,...,

In [208]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)
y = pd.DataFrame(dataset7.iloc[:,-2:])
X

,RH(t-192),Temp(t-192),SH(t-192),sin_time(t-192),cos_time(t-192),GHI(t-192),TARGET(t-192),RH(t-191),Temp(t-191),SH(t-191),sin_time(t-191),cos_time(t-191),GHI(t-191),TARGET(t-191),RH(t-190),Temp(t-190),SH(t-190),sin_time(t-190),cos_time(t-190),GHI(t-190),TARGET(t-190),RH(t-189),Temp(t-189),SH(t-189),sin_time(t-189),cos_time(t-189),GHI(t-189),TARGET(t-189),RH(t-188),Temp(t-188),SH(t-188),sin_time(t-188),cos_time(t-188),GHI(t-188),TARGET(t-188),RH(t-187),Temp(t-187),SH(t-187),sin_time(t-187),cos_time(t-187),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),Temp(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),Temp(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),Temp(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),Temp(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),Temp(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
192,0.665404,0.129630,0.083333,0.500000,1.000000,0.0,0.0,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,...,0.083333,0.146447,0.853553,0.0,0.0,0.649497,0.259259,0.083333,0.195619,0.896677,0.0,0.0,0.695704,0.259259,0.083333,0.250000,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0
193,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,...,0.083333,0.195619,0.896677,0.0,0.0,0.695704,0.259259,0.083333,0.250000,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0
194,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,...,0.083333,0.250000,0.933013,0.0,0.0,0.695379,0.259259,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0
195,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,...,0.083333,0.308658,0.961940,0.0,0.0,0.698842,0.259259,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0,0.698517,0.259259,0.083333,0.629410,0.982963,0.0,0.0
196,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,0.0,0.0,0.725246,0.148148,0.083333,0.961940,0.691342,...,0.083333,0.370590,0.982963,0.0,0.0,0.698950,0.259259,0.083333,0.434737,0.995722,0.0,0.0,0.698085,0.259259,0.083333,0.500000,1.000000,0.0,0.0,0.698085,0.259259,0.083333,0.565263,0.995722,0.0,0.0,0.698517,0.259259,0.083333,0.629410,0.982963,0.0,0.0,0.698193,0.259259

In [209]:
X_train, X_valid, Y_train,  Y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [210]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [211]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [212]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [213]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [214]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [215]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [216]:
testset6 = testset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
testset6.head(10)

,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,70.34,-6.1,9.5,0.000000,1.000000,0.0,0.0
1,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0
2,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0
3,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0
4,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0
5,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0
6,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0
7,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0
8,68.56,-9.1,9.5,0.866025,0.500000,0.0,0.0
9,70.75,-9.5,9.5,0.923880,0.382683,0.0,0.0


In [217]:
testset6 = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)

In [218]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,RH(t-192),T(t-192),SH(t-192),sin_time(t-192),cos_time(t-192),GHI(t-192),TARGET(t-192),RH(t-191),T(t-191),SH(t-191),sin_time(t-191),cos_time(t-191),GHI(t-191),TARGET(t-191),RH(t-190),T(t-190),SH(t-190),sin_time(t-190),cos_time(t-190),GHI(t-190),TARGET(t-190),RH(t-189),T(t-189),SH(t-189),sin_time(t-189),cos_time(t-189),GHI(t-189),TARGET(t-189),RH(t-188),T(t-188),SH(t-188),sin_time(t-188),cos_time(t-188),GHI(t-188),TARGET(t-188),RH(t-187),T(t-187),SH(t-187),sin_time(t-187),cos_time(t-187),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
192,0.679039,0.238889,0.166667,0.500000,1.000000,0.0,0.0,0.696678,0.233333,0.166667,0.565263,0.995722,0.0,0.0,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.0,0.0,0.575695,0.287037,0.166667,0.195619,0.896677,0.0,0.0,0.570393,0.281481,0.166667,0.250000,0.933013,0.0,0.0,0.584461,0.275926,0.166667,0.308658,0.961940,0.0,0.0,0.582621,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.597771,0.264815,0.166667,0.434737,0.995722,0.0,0.0
193,0.696678,0.233333,0.166667,0.565263,0.995722,0.0,0.0,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.0,0.0,0.570393,0.281481,0.166667,0.250000,0.933013,0.0,0.0,0.584461,0.275926,0.166667,0.308658,0.961940,0.0,0.0,0.582621,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.597771,0.264815,0.166667,0.434737,0.995722,0.0,0.0,0.597554,0.261111,0.166667,0.500000,1.000000,0.0,0.0
194,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.0,0.0,0.584461,0.275926,0.166667,0.308658,0.961940,0.0,0.0,0.582621,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.597771,0.264815,0.166667,0.434737,0.995722,0.0,0.0,0.597554,0.261111,0.166667,0.500000,1.000000,0.0,0.0,0.607943,0.257407,0.166667,0.565263,0.995722,0.0,0.0
195,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,0.0,0.0,0.659777,0.183333,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.0,0.0,0.582621,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.597771,0.264815,0.166667,0.434737,0.995722,0.0,0.0,0.597554,0.261111,0.166667,0.500000,1.000000,0.0,0.0,0.607943,0.257407,0.166667,0.565263,0.995722,0.0,0.0,0.612163,0.253704,0.166667,0.629410,0.982963,0.0,0.0
196,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,0.0,0.0,0.659777,0.183333,0.166667,0.933013,0.750000,0.0,0.0,0.683476,0.175926,0.166667,0.961940,0.691342,...,0.166667,0.370590,0.982963,0.0,0.0,0.597771,0.264815,0.166667,0.434737,0.995722,0.0,0.0,0.597554,0.261111,0.166667,0.500000,1.000000,0.0,0.0,0.607943,0.257407,0.166667,0.565263,0.995722,0.0,0.0,0.612163,0.253704,0.166667,0.629410,0.982963,0.0,0.0,0.617357,0.251852,0.166667,0.691342,0.961940,0.0,0

In [219]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?
testset9

,RH(t-192),T(t-192),SH(t-192),sin_time(t-192),cos_time(t-192),GHI(t-192),TARGET(t-192),RH(t-191),T(t-191),SH(t-191),sin_time(t-191),cos_time(t-191),GHI(t-191),TARGET(t-191),RH(t-190),T(t-190),SH(t-190),sin_time(t-190),cos_time(t-190),GHI(t-190),TARGET(t-190),RH(t-189),T(t-189),SH(t-189),sin_time(t-189),cos_time(t-189),GHI(t-189),TARGET(t-189),RH(t-188),T(t-188),SH(t-188),sin_time(t-188),cos_time(t-188),GHI(t-188),TARGET(t-188),RH(t-187),T(t-187),SH(t-187),sin_time(t-187),cos_time(t-187),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,0.611839,0.151852,0.166667,0.500000,1.000000,0.000000,0.000000,0.634347,0.144444,0.166667,0.565263,0.995722,0.000000,0.000000,0.632724,0.137037,0.166667,0.629410,0.982963,0.000000,0.000000,0.650146,0.131481,0.166667,0.691342,0.961940,0.000000,0.000000,0.636511,0.127778,0.166667,0.750000,0.933013,0.000000,0.000000,0.648198,0.124074,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.000000,0.000000,0.547343,0.140741,0.166667,0.195619,0.896677,0.000000,0.000000,0.536306,0.142593,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000
289,0.634347,0.144444,0.166667,0.565263,0.995722,0.000000,0.000000,0.632724,0.137037,0.166667,0.629410,0.982963,0.000000,0.000000,0.650146,0.131481,0.166667,0.691342,0.961940,0.000000,0.000000,0.636511,0.127778,0.166667,0.750000,0.933013,0.000000,0.000000,0.648198,0.124074,0.166667,0.804381,0.896677,0.000000,0.000000,0.637593,0.120370,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.000000,0.000000,0.536306,0.142593,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000
290,0.632724,0.137037,0.166667,0.629410,0.982963,0.000000,0.000000,0.650146,0.131481,0.166667,0.691342,0.961940,0.000000,0.000000,0.636511,0.127778,0.166667,0.750000,0.933013,0.000000,0.000000,0.648198,0.124074,0.166667,0.804381,0.896677,0.000000,0.000000,0.637593,0.120370,0.166667,0.853553,0.853553,0.000000,0.000000,0.642463,0.118519,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000,0.476031,0.157407,0.166667,0.565263,0.995722,0.000000,0.000000
291,0.650146,0.131481,0.166667,0.691342,0.961940,0.000000,0.000000,0.636511,0.127778,0.166667,0.750000,0.933013,0.000000,0.000000,0.648198,0.124074,0.166667,0.804381,0.896677,0.000000,0.000000,0.637593,0.120370,0.166667,0.853553,0.853553,0.000000,0.000000,0.642463,0.118519,0.166667,0.896677,0.804381,0.000000,0.000000,0.621253,0.116667,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000,0.476031,0.157407,0.166667,0.565263,0.995722,0.000000,0.000000,0.452116,0.161111,0.166667,0.629410,0.982963,0.000000,0.000000
292,0.636511,0.127778,0.166667,0.750000,0.933013,0.000000,0.000000,0.648198,0.124074,0.166667,0.804381,0.896677,0.000000,0.000000,0.637593,0.120370,0.166667,0.853553,0.853553,

In [220]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'

    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset2.rename(columns={'T' : 'Temp'}, inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['RH', 'Temp', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] #'DHI', 'DNI', 'season'
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-192),Temp(t-192),SH(t-192),sin_time(t-192),cos_time(t-192),GHI(t-192),TARGET(t-192),RH(t-191),Temp(t-191),SH(t-191),sin_time(t-191),cos_time(t-191),GHI(t-191),TARGET(t-191),RH(t-190),Temp(t-190),SH(t-190),sin_time(t-190),cos_time(t-190),GHI(t-190),TARGET(t-190),RH(t-189),Temp(t-189),SH(t-189),sin_time(t-189),cos_time(t-189),GHI(t-189),TARGET(t-189),RH(t-188),Temp(t-188),SH(t-188),sin_time(t-188),cos_time(t-188),GHI(t-188),TARGET(t-188),RH(t-187),Temp(t-187),SH(t-187),sin_time(t-187),cos_time(t-187),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),Temp(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),Temp(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),Temp(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),Temp(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),Temp(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,0.353641,0.409259,0.166667,0.500000,1.000000,0.0,0.0,0.356671,0.407407,0.166667,0.565263,0.995722,0.0,0.0,0.353641,0.405556,0.166667,0.629410,0.982963,0.0,0.0,0.356780,0.403704,0.166667,0.691342,0.961940,0.0,0.0,0.370306,0.400000,0.166667,0.750000,0.933013,0.0,0.0,0.373444,0.398148,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.000000,0.000000,0.813224,0.303704,0.166667,0.195619,0.896677,0.000000,0.000000,0.805216,0.303704,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.0,0.0,0.798507,0.303704,0.166667,0.370590,0.982963,0.0,0.0,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0
289,0.356671,0.407407,0.166667,0.565263,0.995722,0.0,0.0,0.353641,0.405556,0.166667,0.629410,0.982963,0.0,0.0,0.356780,0.403704,0.166667,0.691342,0.961940,0.0,0.0,0.370306,0.400000,0.166667,0.750000,0.933013,0.0,0.0,0.373444,0.398148,0.166667,0.804381,0.896677,0.0,0.0,0.385240,0.396296,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.000000,0.000000,0.805216,0.303704,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.0,0.0,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0
290,0.353641,0.405556,0.166667,0.629410,0.982963,0.0,0.0,0.356780,0.403704,0.166667,0.691342,0.961940,0.0,0.0,0.370306,0.400000,0.166667,0.750000,0.933013,0.0,0.0,0.373444,0.398148,0.166667,0.804381,0.896677,0.0,0.0,0.385240,0.396296,0.166667,0.853553,0.853553,0.0,0.0,0.391841,0.392593,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0,0.800130,0.298148,0.166667,0.565263,0.995722,0.0,0.0
291,0.356780,0.403704,0.166667,0.691342,0.961940,0.0,0.0,0.370306,0.400000,0.166667,0.750000,0.933013,0.0,0.0,0.373444,0.398148,0.166667,0.804381,0.896677,0.0,0.0,0.385240,0.396296,0.166667,0.853553,0.853553,0.0,0.0,0.391841,0.392593,0.166667,0.896677,0.804381,0.0,0.0,0.406558,0.390741,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.000000,0.000000,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0,0.800130,0.298148,0.166667,0.565263,0.995722,0.0,0.0,0.782599,0.296296,0.166667,0.629410,0.982963,0.0,0.0
292,0.370306,0.400000,0.166667,0.750000,0.933013,0.0,0.0,0.373444,0.398148,0.166667,0.804381,0.896677,0.0,0.0,0.385240,0.396296,0.166667,0.853553,0.853553,0.0,0.0,0.391841,0.392593,0.166667,0.896677,0.804381,0.0,0.0,0.406558,0.390741,0.166667,0.933013,0.750000,0.0,0.0,0.413592,0.387037,0.166667,0.961940,0.691342,...,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.000000,0.000000,0.793529,0.300000,0.166667,0.500000,

In [221]:
Y_train1 = pd.DataFrame(Y_train.iloc[:,0])
Y_train2 = pd.DataFrame(Y_train.iloc[:,1])

Y_valid1 = pd.DataFrame(Y_valid.iloc[:,0])
Y_valid2 = pd.DataFrame(Y_valid.iloc[:,1])

In [222]:
RH_train = X_train.filter(regex='RH')
T_train = X_train.filter(regex='Temp')
# SH_train = X_train.filter(regex='SH')
SH_train = X_train.filter(regex='SH').iloc[:,[0,47,95,143]]
sin_train = X_train.filter(regex='sin_time')
cos_train = X_train.filter(regex='cos_time')
GHI_train = X_train.filter(regex='GHI')
TARGET_train = X_train.filter(regex='TARGET')

RH_valid = X_valid.filter(regex='RH')
T_valid = X_valid.filter(regex='Temp')
# SH_valid = X_valid.filter(regex='SH')
SH_valid = X_valid.filter(regex='SH').iloc[:,[0,47,95,143]]
sin_valid = X_valid.filter(regex='sin_time')
cos_valid = X_valid.filter(regex='cos_time')
GHI_valid = X_valid.filter(regex='GHI')
TARGET_valid = X_valid.filter(regex='TARGET')

RH_test = X_test.filter(regex='RH')
T_test = X_test.filter(regex='Temp')
# SH_test = X_test.filter(regex='SH')
SH_test = X_test.filter(regex='SH').iloc[:,[0,47,95,143]]
sin_test = X_test.filter(regex='sin_time')
cos_test = X_test.filter(regex='cos_time')
GHI_test = X_test.filter(regex='GHI')
TARGET_test = X_test.filter(regex='TARGET')

In [223]:
def df_slicer(data):
  temp = pd.DataFrame()
  for i in range(1,days + 1):
    a = i * 48
    temp2 = data.iloc[:,a-6:a+6]
    temp = pd.concat([temp, temp2], axis =1)
  return temp

In [224]:
RH_train2 = df_slicer(RH_train)
T_train2 = df_slicer(T_train)
# SH_train2 = X_train.filter(regex='SH')
SH_train2 = SH_train.copy()
sin_train2 = df_slicer(sin_train)
cos_train2 = df_slicer(cos_train)
GHI_train2 = df_slicer(GHI_train)
TARGET_train2 = df_slicer(TARGET_train)

RH_valid2 = df_slicer(RH_valid)
T_valid2 = df_slicer(T_valid)
# SH_valid2 = X_valid.filter(regex='SH')
SH_valid2 = SH_valid.copy()
sin_valid2 = df_slicer(sin_valid)
cos_valid2 = df_slicer(cos_valid)
GHI_valid2 = df_slicer(GHI_valid)
TARGET_valid2 = df_slicer(TARGET_valid)

RH_test2 = df_slicer(RH_test)
T_test2 = df_slicer(T_test)
# SH_test2 = X_test.filter(regex='SH')
SH_test2 = SH_test.copy()
sin_test2 = df_slicer(sin_test)
cos_test2 = df_slicer(cos_test)
GHI_test2 = df_slicer(GHI_test)
TARGET_test2 = df_slicer(TARGET_test)

In [225]:
trainset = pd.concat([RH_train2, T_train2, sin_train2, cos_train2, GHI_train2, TARGET_train2, SH_train2], axis = 1)
validset = pd.concat([RH_valid2, T_valid2, sin_valid2, cos_valid2, GHI_valid2, TARGET_valid2, SH_valid2], axis = 1)
testset = pd.concat([RH_test2, T_test2, sin_test2, cos_test2, GHI_test2, TARGET_test2, SH_test2], axis = 1)

In [226]:
testset

,RH(t-150),RH(t-149),RH(t-148),RH(t-147),RH(t-146),RH(t-145),RH(t-144),RH(t-143),RH(t-142),RH(t-141),RH(t-140),RH(t-139),RH(t-102),RH(t-101),RH(t-100),RH(t-99),RH(t-98),RH(t-97),RH(t-96),RH(t-95),RH(t-94),RH(t-93),RH(t-92),RH(t-91),RH(t-54),RH(t-53),RH(t-52),RH(t-51),RH(t-50),RH(t-49),RH(t-48),RH(t-47),RH(t-46),RH(t-45),RH(t-44),RH(t-43),RH(t-6),RH(t-5),RH(t-4),RH(t-3),...,TARGET(t-144),TARGET(t-143),TARGET(t-142),TARGET(t-141),TARGET(t-140),TARGET(t-139),TARGET(t-102),TARGET(t-101),TARGET(t-100),TARGET(t-99),TARGET(t-98),TARGET(t-97),TARGET(t-96),TARGET(t-95),TARGET(t-94),TARGET(t-93),TARGET(t-92),TARGET(t-91),TARGET(t-54),TARGET(t-53),TARGET(t-52),TARGET(t-51),TARGET(t-50),TARGET(t-49),TARGET(t-48),TARGET(t-47),TARGET(t-46),TARGET(t-45),TARGET(t-44),TARGET(t-43),TARGET(t-6),TARGET(t-5),TARGET(t-4),TARGET(t-3),TARGET(t-2),TARGET(t-1),SH(t-192),SH(t-145),SH(t-97),SH(t-49)
288,0.494535,0.490315,0.508062,0.503084,0.522238,0.530246,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.600909,0.627097,0.600260,0.621253,0.573747,0.583811,0.537388,0.537388,0.515853,0.515853,0.504924,0.500487,0.547668,0.542907,0.526350,0.521805,0.504166,0.507845,0.489557,0.506980,0.489016,0.497782,0.487393,0.487393,0.813224,0.813224,0.805216,0.805216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.166667
289,0.490315,0.508062,0.503084,0.522238,0.530246,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.627097,0.600260,0.621253,0.573747,0.583811,0.537388,0.537388,0.515853,0.515853,0.504924,0.500487,0.506655,0.542907,0.526350,0.521805,0.504166,0.507845,0.489557,0.506980,0.489016,0.497782,0.487393,0.487393,0.492587,0.813224,0.805216,0.805216,0.798507,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.166667
290,0.508062,0.503084,0.522238,0.530246,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.600260,0.621253,0.573747,0.583811,0.537388,0.537388,0.515853,0.515853,0.504924,0.500487,0.506655,0.497782,0.526350,0.521805,0.504166,0.507845,0.489557,0.506980,0.489016,0.497782,0.487393,0.487393,0.492587,0.488259,0.805216,0.805216,0.798507,0.805108,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.166667
291,0.503084,0.522238,0.530246,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.621253,0.573747,0.583811,0.537388,0.537388,0.515853,0.515853,0.504924,0.500487,0.506655,0.497782,0.508928,0.521805,0.504166,0.507845,0.489557,0.506980,0.489016,0.497782,0.487393,0.487393,0.492587,0.488259,0.498214,0.805216,0.798507,0.805108,0.793529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.166667
292,0.522238,0.530246,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.573747,0.583811,0.537388,0.537388,0.515853,0.515853,0.504924,0.500487,0.506655,0.497782,0.508928,0.505140,0.504166,0.507845,0.489557,0.506980,0.489016,0.497782,0.487393,0.487393,0.492587,0.488259,0.498214,0.493886,0.798507,0.805108,0.793529,0.800130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [231]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=20000, bagging_fraction=0.7, learning_rate=0.01, save_binary = True, subsample=0.7, device = 'gpu')          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [232]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [233]:
qq = 0
quantiles = [0.1-qq, 0.2-qq, 0.3-qq, 0.4-qq, 0.5, 0.6-qq, 0.7-qq, 0.8-qq, 0.9-qq]

In [234]:
tick = time.time()

# Target1

models_1, results_1 = train_data(trainset, Y_train1, validset, Y_valid1, testset)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.40162
[1000]	valid_0's quantile: 1.37899
[1500]	valid_0's quantile: 1.37062
[2000]	valid_0's quantile: 1.36607
[2500]	valid_0's quantile: 1.36481
[3000]	valid_0's quantile: 1.36288
[3500]	valid_0's quantile: 1.36082
[4000]	valid_0's quantile: 1.36028
[4500]	valid_0's quantile: 1.35906
[5000]	valid_0's quantile: 1.35876
[5500]	valid_0's quantile: 1.36074
Early stopping, best iteration is:
[5321]	valid_0's quantile: 1.35803
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.21753
[1000]	valid_0's quantile: 2.17049
[1500]	valid_0's quantile: 2.14327
[2000]	valid_0's quantile: 2.1207
[2500]	valid_0's quantile: 2.10777
[3000]	valid_0's quantile: 2.09889
[3500]	valid_0's quantile: 2.09065
[4000]	valid_0's quantile: 2.08192
[4500]	valid_0's qu

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [235]:
# Target2

models_2, results_2 = train_data(trainset, Y_train2, validset, Y_valid2, testset)
tock = time.time()
results_2.sort_index()

0.1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.40321
[1000]	valid_0's quantile: 1.38587
[1500]	valid_0's quantile: 1.38275
[2000]	valid_0's quantile: 1.38047
[2500]	valid_0's quantile: 1.37662
[3000]	valid_0's quantile: 1.37651
[3500]	valid_0's quantile: 1.37567
[4000]	valid_0's quantile: 1.37488
[4500]	valid_0's quantile: 1.37515
Early stopping, best iteration is:
[4233]	valid_0's quantile: 1.37444
0.2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.7 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.26506
[1000]	valid_0's quantile: 2.20747
[1500]	valid_0's quantile: 2.17817
[2000]	valid_0's quantile: 2.16091
[2500]	valid_0's quantile: 2.14765
[3000]	valid_0's quantile: 2.13989
[3500]	valid_0's quantile: 2.13015
[4000]

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [236]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 2134.687031507492
(3888, 9) (3888, 9)


In [237]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [238]:
submission3.to_csv('submission_resize4.csv')
!cp submission_resize4.csv "drive/My Drive/"